Αικατερίνη Δρακάκη <br>
1115201300041 <br>
Νικόλαος Μακρυγεώργος <br>
1115201500238

*****
*****
# Vectorization - εξαγωγή χαρακτηριστικών
# Προσθήκη Χαρακτηριστικών
# SVM  -  KNN   -  ταξινομητές
*****
*****

In [1]:
import nltk
import re
import pickle
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
from pandas import DataFrame
from collections import Counter
#nltk.download('all')
import pandas as pd
import sys
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import random as random
import combi
from combi import *

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
warnings.filterwarnings("ignore", category=DeprecationWarning)

import gensim
from gensim.models import Word2Vec 
from gensim.test.utils import common_texts, get_tmpfile
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.manifold import TSNE

from sklearn import metrics
from nltk import sent_tokenize
from random import randint

from nltk import word_tokenize
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import StemmerI, RegexpStemmer, LancasterStemmer, ISRIStemmer, PorterStemmer, SnowballStemmer, RSLPStemmer

Φτιαχνουμε λίστες με τα train tweets και τα αποτελέσματα τους.

In [2]:
small_character_tweets = ""
results = []
stop__words = set(stopwords.words('english'))

with open("train2017.tsv", encoding="utf8" ) as file_in_put:
    for line in file_in_put:
        field = line.split()
        results.append(field[2]);
        line = line.lower()
        list = []
        list = line.split()
        for i in range(len(list)):
            if i > 2 and list[i] not in stop__words and len(list[i]) > 1:
                small_character_tweets+=list[i]     
                small_character_tweets+=" "                
        small_character_tweets+="\n"
    del list[:]

results.pop(25108)
#print (len(results))

'neutral'

****

****

Φτιάχνουμε μια λίστα με τα σωστά αποτελέσματα απο τα test δεδομένα για να τα συγκρίνουμε με τα αποτελέσματα των SVM - KNN <br>
και να βγάλουμε στατιστικά

In [3]:
test_results = []

with open("SemEval2017_task4_subtaskA_test_english_gold.txt", encoding="utf8" ) as test_results__:
    for line in test_results__:
        field = line.split()
        test_results.append(field[1])

#print (len(test_results))

Φτιάχνουμε δυο λίστες με τα test δεδομένα.Η λίστα test_tweetList περιέχει κάθε tweet ενω η λίστα tokenized_test_tweets περιέχει <br>
λίστες με τις λέξεις κάθε tweet.

Επίσης κάνουμε ένα μικρό καθαρισμό των tweets

In [4]:
str_tweets = ""
with open("test2017.tsv", encoding="utf8" ) as test_tweets__:
    for line in test_tweets__:
        field = line.split()
        line = line.lower()
        list = []
        list = line.split()
        for i in range(len(list)):
            if i > 2 and list[i] not in stop__words and len(list[i]) > 1:
                str_tweets+=list[i]
                str_tweets+=" "
        str_tweets+="\n"
        del list[:]
        
str_tweets = str_tweets.replace('#','').replace('@','').replace('-',' ').replace('@','').replace('!','').replace('?','')
str_tweets = str_tweets.replace(',',' ').replace('.','').replace('*',' ').replace(':',' ').replace('(',' ').replace(')',' ')

test_tweetList = []
test_tweetList = str_tweets.split("\n")
test_tweetList.pop(12284)
#print (len(test_tweetList))

tokenized_test_tweets = []
for i in test_tweetList:
    f = i.split()
    tokenized_test_tweets.append(f)

#print (len(tokenized_test_tweets))

***

***

Παρακάτω κάνουμε τις ίδιες διαδικασίες για τα test tweets.

In [5]:
small_character_tweets = small_character_tweets.replace(',','').replace('-','').replace('_','').replace('<','')
small_character_tweets = small_character_tweets.replace('>','').replace('"  "" "', '').replace('#','')
small_character_tweets = small_character_tweets.replace('&','').replace('+','').replace('=','').replace('/','')
small_character_tweets = small_character_tweets.replace('(','').replace(')','').replace(':','').replace('!','')
small_character_tweets = small_character_tweets.replace('?','').replace(';','').replace('$','')
small_character_tweets = small_character_tweets.replace("'m"," am").replace('.','')
small_character_tweets = small_character_tweets.replace('"\"','').replace('\\','').replace('\'s','').replace('\'','')

small_character_tweets = small_character_tweets.replace('@','').replace('u2019',' ').replace('u002c','')

In [6]:
tweets2 = ""
tweets2 = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9_]+)', r' user ', small_character_tweets)
tweets2 = ' '.join(word for word in tweets2.split(' ') if not word.startswith('http'))

tweetList = []
tweetList = tweets2.split('\n')     
tweetList.pop(len(tweetList)-1)
tweetList.pop(25108)
#print (len(tweetList))

tokenized_tweets = []
for i in tweetList:
    f = i.split()
    tokenized_tweets.append(f)

***********
            Bag of Words
            Πρώτα για τα train και μετα για τα test.
***********

In [7]:
vectorizer_Bag = CountVectorizer(max_features=1000)
X1 = vectorizer_Bag.fit_transform(tweetList)
#print(vectorizer_Bag.get_feature_names())
#print(X1.toarray())
#print (X1.shape)
tweet_list_vector_BGW = X1.toarray()
#print (len(tweet_list_vector_BGW))

In [8]:
vectorizer_Bag_test = CountVectorizer(max_features=1000)
X11 = vectorizer_Bag_test.fit_transform(test_tweetList)

#print (X11.shape)
tweet_list_vector_BGW_test = X11.toarray()
#print (len(tweet_list_vector_BGW_test))

**********
             Tf idf
             Πρώτα για τα train και μετα για τα test.
**********

In [9]:
vectorizer_TF = TfidfVectorizer(max_features=1000)
X2 = vectorizer_TF.fit_transform(tweetList)
#print(vectorizer_TF.get_feature_names())
#print(X2.shape)
#print(X2.toarray())
tweet_list_vector_TFIDF = X2.toarray()
#print (len(tweet_list_vector_TFIDF))

In [10]:
vectorizer_TF_test = TfidfVectorizer(max_features=1000)
X22 = vectorizer_TF_test.fit_transform(test_tweetList)

tweet_list_vector_TFIDF_test = X22.toarray()
#print (len(tweet_list_vector_TFIDF_test))

*****
             Word Embeddings
             Πρώτα για τα train και μετα για τα test.
*****
Παρακάτω η μέθοδος Word2vec

In [11]:
#model_w2v = gensim.models.Word2Vec(tokenized_tweets,size=200,window=5,min_count=2)
#model_w2v.train(tokenized_tweets, total_examples= len(tokenized_tweets), epochs=20)   
#model_w2v.save("apotelesmata_train_gia1000.bin")
#model_w2v = Word2Vec.load("apotelesmata_train_gia1000.bin")

In [12]:
#test_model_w2v = gensim.models.Word2Vec(tokenized_test_tweets,size=200,window=5,min_count=2)
#test_model_w2v.save("apotelesmata_test_gia1000.bin")
#test_model_w2v = Word2Vec.load("apotelesmata_test_gia1000.bin")

Επειδή δεν έιχαμε καλά αποτελέσματα με τα διανύσματα απο το Work2vec χρησιμοποιούμε τα έτοιμα διανύσματα που δόθηκαν. <br>
Αρα διαβάζουμε απο το αρχείο και φτιάχνουμε δυο λίστες,η μία με τις λέξεις και η άλλη με το διάνυσμα καθε λέξης.

In [13]:
wordss = []
vectors = []

with open("datastories.twitter.200d.txt", encoding="utf8" ) as words:
    for line in words:
        A = line.split()
        wordss.append(A[0])
        A.pop(0)
        vectors.append(A)
     
#print (len(wordss))
#print (len(vectors))

Η συνάρτηση mean2 φτιάχνει για κάθε tweet ένα διάνυσμα.Το διάνυσμα κάθε λέξης είναι αυτό απο το αρχείο.

In [14]:
def mean2(tweetList,words,vectors):
    meanList_id = []
    meanList_vec = []
    id = 1
    for t in tweetList:
        meanList_id.append(id)
        id+=1
        vecList = []
        sum = 0
        for j in range(0,200):
            vecList.append(0)
            
            
            
        tokens = t.split()   
        for i in range(len(tokens)):
            sum+=1
            if tokens[i] in words:
                
                for k in range(len(words)):
                    if tokens[i] == words[k]:
                        vec = vectors[k]
                        for j in range(0,200):
                            vecList[j] = vecList[j] + float(vec[j])
                        break
                        
            else:
                for j in range(0,200):
                    vecList[j] = vecList[j] + random.uniform(-1, 1)
        
        
        for j in range(0,200):
            vecList[j] = vecList[j]/sum
        meanList_vec.append(vecList)
        
    return meanList_id, meanList_vec



tweet_list_id, tweet_list_vector_WEM = mean2(tweetList,wordss,vectors)
test_tweet_id, test_tweet_WEM = mean2(test_tweetList,wordss,vectors)

Ιδια διαδικασία με την παραπάνω αλλά για την μέθοδο Word2vec

In [15]:

def mean(tweetList,model_w2v):
    wordModel = model_w2v.wv
    meanList_id = []
    meanList_vec = []
    id = 1
    for t in tweetList:
        meanList_id.append(id)
        id+=1
        vecList = []
        sum = 0
        for j in range(0,200):
            vecList.append(0)
    
    
    
        tokens = t.split()
        for i in range(len(tokens)):
            sum+=1
            if tokens[i] in wordModel.vocab:
                vector = model_w2v[tokens[i]]
                for j in range(0,200):
                    vecList[j] = vecList[j] + vector[j]
            else:
                for j in range(0,200):
                    vecList[j] = vecList[j] + random.uniform(-1, 1)
    
    
        for j in range(0,200):
            vecList[j] = vecList[j]/sum
        meanList_vec.append(vecList)

        
    return meanList_id, meanList_vec

#tweet_list_id, tweet_list_vector_WEM = mean(tweetList,model_w2v)
#test_tweet_id, test_tweet_WEM = mean(test_tweetList,test_model_w2v)

Αντικαθιστούμε τα συναισθήματα με 0,1,2
Δουλεύει και χωρίς αυτό.

In [16]:
# negative = 0
# neutral = 1
# positive = 2

#print (results)
#for i in range(len(results)):
#    if results[i] == "negative":
#        results[i] = 0
#    elif results[i] == "neutral":
#        results[i] = 1
#    else:
#        results[i] = 2
#print (results)    

In [17]:
#for i in range(len(test_results)):
#    if test_results[i] == "negative":
#        test_results[i] = 0
#    elif test_results[i] == "neutral":
#        test_results[i] = 1
#    else:
#        test_results[i] = 2

****
     Ταξινομητής   SVM 
****

In [18]:
# SVM - Bag of Words

svc_BOW = svm.SVC(kernel='linear', C=1, probability=True)  
svc_BOW = svc_BOW.fit(tweet_list_vector_BGW , results)
results_BOW = svc_BOW.predict(tweet_list_vector_BGW_test) 
f1_score(test_results, results_BOW, average='macro')

0.3697537528373704

In [19]:
# SVM - Tf Idf

svc_TfIdf = svm.SVC(kernel='linear', C=1, probability=True)
svc_TfIdf = svc_TfIdf.fit(tweet_list_vector_TFIDF , results)
results_TfIdf = svc_TfIdf.predict(tweet_list_vector_TFIDF_test)
f1_score(test_results, results_TfIdf, average='macro') 

0.33729129033705113

In [20]:
# SVM - Word Embeddings

svc_WEMBEDDINGS = svm.SVC(kernel='linear', C=1, probability=True) 
svc_WEMBEDDINGS = svc_WEMBEDDINGS.fit(tweet_list_vector_WEM, results)
results_WEMBEDDINGS = svc_WEMBEDDINGS.predict(test_tweet_WEM)
f1_score(test_results, results_WEMBEDDINGS, average='macro')

0.5670108940179143

***
     Ταξινομητής   ΚΝΝ
***

In [21]:
# KNN - Bag of Words

knn_BOW = KNeighborsClassifier(n_neighbors=100)
knn_BOW.fit(tweet_list_vector_BGW , results)
results_BOW = knn_BOW.predict(tweet_list_vector_BGW_test)
print(metrics.accuracy_score(test_results, results_BOW))

0.4832302181699772


In [22]:
# KNN - Tf Idf 

knn_TfIdf = KNeighborsClassifier(n_neighbors=100) 
knn_TfIdf.fit(tweet_list_vector_TFIDF , results)
results_TfIdf = knn_TfIdf.predict(tweet_list_vector_TFIDF_test)
print(metrics.accuracy_score(test_results, results_TfIdf))

0.4826603712145881


In [23]:
# KNN - Word Embeddings

knn_WEMBEDDINGS = KNeighborsClassifier(n_neighbors=150)
knn_WEMBEDDINGS.fit(tweet_list_vector_WEM , results)
results_WEMBEDDINGS = knn_WEMBEDDINGS.predict(test_tweet_WEM)
print(metrics.accuracy_score(test_results, results_WEMBEDDINGS))

0.572614783458157


****
****
****
Παρακάτω κάνουμε ΚΝΝ και SVM με προσθήκη χαρακτηριστικών.

Βάζουμε σε λίστες κάθε λεξικό μαζί με το συναίσθημα του.

In [24]:
lex1 = []
lex2 = []
lex3 = []
lex4 = []
lex5 = []

def lexica_in_list(file):
    lexica = []
    with open(file, encoding="utf8") as f:
        for lines in f:
            k = lines.split()
            length = len(k)
            str = ""
            for i in range(len(k) -1):
                if i > 0:
                    str+= " "
                str+= k[i]
            lexica.append(str)
            lexica.append(k[length-1])
            
    return lexica

lex1 = lexica_in_list("affin.txt")
lex2 = lexica_in_list("valence_tweet.txt")
lex3 = lexica_in_list("generic.txt")
lex4 = lexica_in_list("nrc.txt")
lex5 = lexica_in_list("nrctag.txt")

Υπολογίζουμε τα νέα χαρακτηριστικά για κάθε tweet.Αυτά είναι : <br>
το μικρότερο συναίσθημα του tweet <br>
το μεγαλύτερο συναίσθημα του tweet <br>
ο μέσος όρος συναισθημάτων <br>
Αυτη η διαδικασία γίνεται για κάθε λεξικό. <br>
Αν κάποια λέξη δεν υπάρχει στο λεξικό τότε παίρνει τυχαία τιμή συναισθήματος. <br> 
<br>
H παρακάτω διαδικασία είναι πολύ χρονοβόρα για το λόγο αυτό τα αποθηκεύουμε και τα κάνουμε την επόμενη φορά load.

In [25]:
l1 = []
l2 = []
l3 = []
l4 = []
l5 = []

def new_features(tweetList, lex, kk):
    l2 = []
    for tweet in tweetList:
        l = []
        t = tweet.split()
        min = 1000
        max = -1000
        sum = 0
        for i in range(len(t)):
            find= 0
            for j in range(len(lex))[::2]:
                if t[i] == lex[j]:
                    find = 1
                    if float(lex[j+1]) < min:
                        min = float(lex[j+1])
                    if float(lex[j+1]) > max:
                        max = float(lex[j+1])
                    sum+=float(lex[j+1])
                    break
            if find == 0:
                if kk == 1:
                    word = randint(-3,3)
                elif kk == 2:
                    word = random.uniform(-1, 1)
                elif kk == 3:
                    word = random.uniform(-0.9, 0.9)
                else:
                    word = random.uniform(-5, 5)
                if word < min:
                    min = word
                if word > max:
                    max = word
                sum+=word
                    
        sum = sum/len(t)
        l.append(min)
        l.append(max)
        l.append(sum)
        l2.append(l)

    return l2
    
#l1 = new_features(tweetList, lex1, 1)
#l2 = new_features(tweetList, lex2, 2)
#l3 = new_features(tweetList, lex3, 3)
#l4 = new_features(tweetList, lex4, 4)
#l5 = new_features(tweetList, lex5, 5)

#with open("l1.txt", "wb") as fp1:
    #pickle.dump(l1, fp1)
#with open("l2.txt", "wb") as fp2:
    #pickle.dump(l2, fp2)
#with open("l3.txt", "wb") as fp3:
 #   pickle.dump(l3, fp3)
#with open("l4.txt", "wb") as fp4:
    #pickle.dump(l4, fp4)
#with open("l5.txt", "wb") as fp5:
    #pickle.dump(l5, fp5)

Φόρτωση χαρακτηριστηκών για κάθε tweet και κάθε λεξικό σε λίστες.

In [26]:
with open("l1.txt", "rb") as fp1:
    l1 = pickle.load(fp1) 
with open("l2.txt", "rb") as fp2:
    l2 = pickle.load(fp2)
with open("l3.txt", "rb") as fp3:
    l3 = pickle.load(fp3)
with open("l4.txt", "rb") as fp4:
    l4 = pickle.load(fp4)
with open("l5.txt", "rb") as fp5:
    l5 = pickle.load(fp5)

Δημιουργία χαρακτηριστικών για τα test δεδομένα

In [27]:
#l1_test = new_features(test_tweetList, lex1, 1)
#l2_test = new_features(test_tweetList, lex2, 2)
#l3_test = new_features(test_tweetList, lex3, 3)
#l4_test = new_features(test_tweetList, lex4, 4)
#l5_test = new_features(test_tweetList, lex5, 5)

#with open("l1_test.txt", "wb") as t1:
    #pickle.dump(l1_test, t1)
#with open("l2_test.txt", "wb") as t2:
  #  pickle.dump(l2_test, t2)
#with open("l3_test.txt", "wb") as t3:
  #  pickle.dump(l3_test, t3)
#with open("l4_test.txt", "wb") as t4:
    #pickle.dump(l4_test, t4)
#with open("l5_test.txt", "wb") as t5:
    #pickle.dump(l5_test, t5)

Φόρτωση χαρακτηριστηκών για κάθε tweet και κάθε λεξικό σε λίστες για τα test δεδομένα.

In [28]:
with open("l1_test.txt", "rb") as t1:
    l1_test = pickle.load(t1)
with open("l2_test.txt", "rb") as t2:
    l2_test = pickle.load(t2)
with open("l3_test.txt", "rb") as t3:
    l3_test = pickle.load(t3)
with open("l4_test.txt", "rb") as t4:
    l4_test = pickle.load(t4)
with open("l5_test.txt", "rb") as t5:
    l5_test = pickle.load(t5)

Τα νέα χαρακτηριστικά τα προσθέτουμε στα διανύσματα που ήδη υπάρχουν.

In [29]:
for i in range(len(tweet_list_vector_WEM)):
    tweet_list_vector_WEM[i].append(l1[i][0])
    tweet_list_vector_WEM[i].append(l1[i][1])
    tweet_list_vector_WEM[i].append(l1[i][2])
    
    tweet_list_vector_WEM[i].append(l2[i][0])
    tweet_list_vector_WEM[i].append(l2[i][1])
    tweet_list_vector_WEM[i].append(l2[i][2])
    
    tweet_list_vector_WEM[i].append(l3[i][0])
    tweet_list_vector_WEM[i].append(l3[i][1])
    tweet_list_vector_WEM[i].append(l3[i][2])
    
    tweet_list_vector_WEM[i].append(l4[i][0])
    tweet_list_vector_WEM[i].append(l4[i][1])
    tweet_list_vector_WEM[i].append(l4[i][2])
    
    tweet_list_vector_WEM[i].append(l5[i][0])
    tweet_list_vector_WEM[i].append(l5[i][1])
    tweet_list_vector_WEM[i].append(l5[i][2])
    

for i in range(len(test_tweet_WEM)):
    test_tweet_WEM[i].append(l1_test[i][0])
    test_tweet_WEM[i].append(l1_test[i][1])
    test_tweet_WEM[i].append(l1_test[i][2])
    
    test_tweet_WEM[i].append(l2_test[i][0])
    test_tweet_WEM[i].append(l2_test[i][1])
    test_tweet_WEM[i].append(l2_test[i][2])
    
    test_tweet_WEM[i].append(l3_test[i][0])
    test_tweet_WEM[i].append(l3_test[i][1])
    test_tweet_WEM[i].append(l3_test[i][2])
    
    test_tweet_WEM[i].append(l4_test[i][0])
    test_tweet_WEM[i].append(l4_test[i][1])
    test_tweet_WEM[i].append(l4_test[i][2])
    
    test_tweet_WEM[i].append(l5_test[i][0])
    test_tweet_WEM[i].append(l5_test[i][1])
    test_tweet_WEM[i].append(l5_test[i][2])

***
 SVM + KNN  για Word Embeddings + new features
***

In [30]:
# SVM - Word Embeddings + new features

svc_WEMBEDDINGS = svm.SVC(kernel='linear', C=1, probability=True) 
svc_WEMBEDDINGS = svc_WEMBEDDINGS.fit(tweet_list_vector_WEM, results)
results_WEMBEDDINGS = svc_WEMBEDDINGS.predict(test_tweet_WEM)
f1_score(test_results, results_WEMBEDDINGS, average='macro')

0.583986854473807

Με αυτα τα 15 χαρακτηριστικά που προσθέσαμε βλέπουμε μια μικρή αύξηση απο 0.56 σε 0.58
****
Για τον ΚΝΝ θα αφήσουμε μόνο τον μέσο ορο δηλαδή θα έχει μόνο 5 επιπλέον χαρακτηριστικά. <br>
Με τα 15 επιπλέον όπως ο SVM είχε αρνητικά αποτελέσματα ετσι τα αφαιρούμε τα 10.

In [70]:
for j in range(len(tweet_list_vector_WEM)):
    tweet_list_vector_WEM[j].pop(200)
    tweet_list_vector_WEM[j].pop(200)  
    tweet_list_vector_WEM[j].pop(201)
    tweet_list_vector_WEM[j].pop(201)
    tweet_list_vector_WEM[j].pop(202)
    tweet_list_vector_WEM[j].pop(202)
    tweet_list_vector_WEM[j].pop(203)
    tweet_list_vector_WEM[j].pop(203)
    tweet_list_vector_WEM[j].pop(204)
    tweet_list_vector_WEM[j].pop(204)
    
for j in range(len(test_tweet_WEM)):
    test_tweet_WEM[j].pop(200)
    test_tweet_WEM[j].pop(200)
    test_tweet_WEM[j].pop(201)
    test_tweet_WEM[j].pop(201)
    test_tweet_WEM[j].pop(202)
    test_tweet_WEM[j].pop(202)
    test_tweet_WEM[j].pop(203)
    test_tweet_WEM[j].pop(203)
    test_tweet_WEM[j].pop(204)
    test_tweet_WEM[j].pop(204)

215
205


In [71]:
# KNN - Word Embeddings + new features

knn_WEMBEDDINGS = KNeighborsClassifier(n_neighbors=150)
knn_WEMBEDDINGS.fit(tweet_list_vector_WEM , results)
results_WEMBEDDINGS = knn_WEMBEDDINGS.predict(test_tweet_WEM)
print(metrics.accuracy_score(test_results, results_WEMBEDDINGS))

0.5653695864539238
